# API requests
#### Preparation stage

In [ ]:
# importing necessary libraries
import pandas as pd
import requests
from datetime import datetime
import pytz

In [ ]:
import sqlalchemy
!pip install pymysql
import pymysql

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
PASSWORD = os.environ.get('PASSWORD1')
API_key1 = os.environ.get('API_key_weather')

In [ ]:
print(API_key1)

b1113302dc4c208d6a0581817568a48e


# Function for updating weather

In [ ]:
def get_city_weather(cities):
    for city in cities:
        # API CALL
        API_key = API_key1 # <----- your API_key
        # parameters
        limit = 5  # I think this arks the days
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
        # END OF API CALL parameters
        ow_5d_fcast = requests.get(url)

        # defining a first dataframe
        weather_df = pd.json_normalize(ow_5d_fcast.json()['list'])
        # ----------------------------------------------
        # defining the last dataframe + cleaning
        weath_dfz = weather_df.loc[:,["dt_txt","main.temp","wind.speed", "main.pressure"]]

        # insert the city name as a column
        weath_dfz.insert(loc=1, column="city",value=city) ###### Value for the city

        # inserting the weather column
        weath_dfz.insert(loc=2, column="weather",value=None) ###### Value for the city

        # preparing input
        # cleaning weather column off list and dictionary into new DF
        weather_list_dict = weather_df['weather'].apply(pd.Series)[0].apply(pd.Series)
        # combining two columns into new Series
        weather_final = weather_list_dict.main + ": "+ weather_list_dict.description
        # adding series column to final df
            # note afterwards
        weath_dfz['weather'] = weather_final.values 
            # because indices don't match we need the .values
        weath_dfz.head(3)

        # adding the rain.3h column in case it is given
        if "rain.3h" in weather_df.columns:
            weath_dfz.insert(loc=3, column="rain_prob",value=weather_df["rain.3h"]) ###### Value for the city
        # weath_dfz.set_index("dt_txt",inplace=True)

        weath_dfz.head(3)

        ### for Testing

        if cities.index(city) ==0:
            weather_DF = weath_dfz
        else:
            weather_DF = pd.concat([weather_DF,weath_dfz], ignore_index=True)

    return weather_DF

## Application of the function with list of cities

In [ ]:
cities = ['Istanbul',
 'Moscow',
 'London',
 'Saint Petersburg',
 'Berlin',
 'Madrid',
 'Kyiv',
 'Rome',
 'Bucharest',
 'Paris']
weather_TABLE = get_city_weather(cities)

## Renaming the function for later reference

In [ ]:
weather_TABLE.iloc[:,0].head(3)

0    2022-11-30 18:00:00
1    2022-11-30 21:00:00
2    2022-12-01 00:00:00
Name: dt_txt, dtype: object

In [ ]:
weather_TABLE.rename(columns = {'dt_txt':'weather_PK'}, inplace = True)

In [ ]:
weather_TABLE.head(3)

,weather_PK,city,weather,rain_prob,main.temp,wind.speed,main.pressure
0,2022-11-30 18:00:00,Istanbul,Clouds: broken clouds,NaN,13.32,6.98,1017
1,2022-11-30 21:00:00,Istanbul,Clouds: overcast clouds,NaN,12.94,6.12,1018
2,2022-12-01 00:00:00,Istanbul,Rain: light rain,0.15,12.67,4.60,1018


## Changing datatype to date

In [ ]:
weather_TABLE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   weather_PK     400 non-null    object 
 1   city           400 non-null    object 
 2   weather        400 non-null    object 
 3   rain_prob      45 non-null     float64
 4   main.temp      400 non-null    float64
 5   wind.speed     400 non-null    float64
 6   main.pressure  400 non-null    int64  
dtypes: float64(3), int64(1), object(3)
memory usage: 22.0+ KB


In [ ]:
weather_TABLE["weather_PK"] = pd.to_datetime(weather_TABLE["weather_PK"])

In [ ]:
weather_TABLE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   weather_PK     400 non-null    datetime64[ns]
 1   city           400 non-null    object        
 2   weather        400 non-null    object        
 3   rain_prob      45 non-null     float64       
 4   main.temp      400 non-null    float64       
 5   wind.speed     400 non-null    float64       
 6   main.pressure  400 non-null    int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 22.0+ KB


## Pushing to mySQL

In [ ]:
# !pip install sqlalchemy 
# import sqlalchemy # install if needed

# !pip install pymysql 
# import pymysql

### Safety - setting password up as environental variable

### Linking to the SQL database with variables

In [ ]:
schema="P3_Cities"   # name of the database you want to use here
host="city-project3-db.cf7wpiecfnwf.us-east-1.rds.amazonaws.com"        # to connect to your local server
user="admin"
password = PASSWORD  # <----- your PASSWORD
port= 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### Before the push 

### When pushing Dataframes to SQL they will become Tables in SQL.
###### When Pushing there is one requirement, the database/schema into the which data is going to be pushed has to be created in the respective instance/connection (to choose from mySQL "home"). There are three possible scenarios for pushing.
```
        P.1.  The Table hasn't been created in the DB: 
                then the push is gonna create it. 
        P.2.  The Table has been created without foreign keys:
                then the push is gonna add informaiton to the table.
        P.3.  The Table has been created with with foreign keys:
                  then the push can only happen, by respecting consitency rules of SQL.
```

### P.1. The Push: creating the table

In [ ]:
weather_TABLE.to_sql('cities_weather',     # table name;
                       if_exists='append',      # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
                      con=con,               # con-> connection string;
                      index=False)           # index = False -> will not send index column to database
                    # I changed this to true so the column stays


400

In [ ]:
weather_TABLE.head(20)